
# **Gestión de datos y datos digitales**
### **Autor: Ferran Carrascosa Mallafrè**

---
---

<!-- script html for image -->

<center>


<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c9/Star_Wars%2C_%C3%A9pisode_III_-_La_Revanche_des_Sith_logo.jpg/220px-Star_Wars%2C_%C3%A9pisode_III_-_La_Revanche_des_Sith_logo.jpg"  width="500" height="150"/>


<br>

Fuente de la imagen: [https://es.wikipedia.org](https://es.wikipedia.org/wiki/Star_Wars:_Episodio_III_-_La_venganza_de_los_Sith)

</center>

<br>

# **Índice**

---
---

> [Gestión de datos y datos digitales](#scrollTo=eJfiC-K0VViA&uniqifier=1)
<br>
>>
>> [Preparación del Entorno](#scrollTo=UkU8akIVtlbn&line=1&uniqifier=1) \\
>>   \\
>> [1.3. Google Analytics](#scrollTo=U3oTMspTVVjE&uniqifier=1) \\
>>
>>> [1.3.1. Google developer console](#scrollTo=Hd88I7fxVVju&uniqifier=1) \\
>>>
>>> [1.3.2. Google Analytics Reports](#scrollTo=yxlPi4fjjYGb&uniqifier=1) \\
>>>
>>> [1.3.3. Acceso a Google Analytics des de Python](#scrollTo=uNp0KfRlVVkP&uniqifier=1)
>>
>> [Enlaces de interés](#scrollTo=C-AMazPq4Htt&uniqifier=1)

# Preparación del entorno

Cargamos las librerías.

In [ ]:
import pandas as pd
import numpy as np
import json
import os
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

# 1.3. Google Analytics


Una de las primeras tareas para crear un nuevo website es configurar Google Analytics. Después del tiempo y esfuerzo invertido en construir una web, habitualmente se quiere medir y analizar el comportamiento de los usuarios.

Las preguntas más frecuentes son:

- ¿Como esta funcionando mis contenidos?
- ¿Qué categorías de productos son los más populares?
- ¿Cuantas conversiones ha realizado mi campaña de Facebook?
- ¿Los usuarios ven el último video que he producido?
- ¿Que comparten los usuarios en las redes sociales?
- ¿Las personas abandonan la pagina durante el proceso de pago?

El [54.6% de todos los websites utilizan Google analytics](https://w3techs.com/technologies/details/ta-googleanalytics)) y el [84% del mercado de herramientas de analisis web](https://w3techs.com/technologies/overview/traffic_analysis) también es de Google Analytics.

El objetivo es poder tomar decisiones basados en datos. Las decisiones pueden ser, por ejemplo, el tipo de contenidos invertir, o qué presupuesto puedes gastar en campañas de anuncios.

Para aprender a configurar Google Analytics una recomendación, es utilizar el curso de [Google Analytics para principiantes](https://analytics.google.com/analytics/academy/course/6) o incluso el [módulo avanzado](https://analytics.google.com/analytics/academy/).

El objetivo del módulo es iniciarse en la conexión a su API de datos para poder extraer información a través de Python. De esta forma, veremos cómo convertir los datos de Google Analytics en tablas pandas. 

> **Importante**: Obviamente, para poder practicar con el contenido de este módulo necessitas de tu propia cuenta de Google.



## 1.3.1. Google developer console

Para acceder puedes acceder al siguiente enlace web: https://console.developers.google.com/. 

Una vez dentro, debes crear un proyecto nuevo. Esto lo puedes hacer en la pestaña que encontrarás en la parte superior izquierda. Clicas en la pestaña y aprietas la opción de **NUEVO PROYECTO**.

Tienes que dar un nombre al proyecto. Por ejemplo "GA master IL3" y le das al botón de crear.

A continuación, aparecerá consola de desarrollo. Debes habilitar las API's: 
-	Google Analytics Reporting API
-	Google Analytics API

Para ello, clicas en la pestaña (izquierda) de la Biblioteca y buscas el nombre de la API (por ejemplo, Google Analytics Reporting API). A continuación, te aparecerá la opción de habilitar.

El Siguiente paso consiste en crear las credenciales de acceso a los datos. Para ello clica en la pestaña (izquierda) de Credenciales. A continuación clica en la pestaña “Crear Credenciales” (superior) y escoges la opción de **Cuenta de Servicio**.

En detalles de la cuenta de servicio pon un nombre de la cuenta. Por ejemplo: ga-il3-test .  Aprietas la opción de **Crear**.

En la caja de Función, otorga a esta cuenta de servicio acceo al proyecto de tipo "Propietario". Puedes buscar o escribirlo directamente en la caja. Dale al Botón de Continuar y finalmente al botón de Listo.

A continuación, e la pestaña de credenciales, revisa que se ha creado la Cuenta de servicio que deberia llevar el siguiente nombre (según lo hayas configurado):   ga-il3-test@ga-master-il3.iam.gserviceaccount.com . Este correo, lo necesitaras mas adelante.

El último paso para obtener las credenciales, es Agregar un clave de acceso a la cuenta de servicio. Para ello, entra en la configuración de la cuenta de servicio clicando sobre el email del nombre de la cuenta. En la siguiente pantalla, clica en a pestaña  de "AGREGAR CLAVE" > "Crear clave nueva". Se abre un emergente con la opción por defecto de JSON. Aprietas en la opción de "CREAR".

En mi caso este proceso devuelve un mensaje de "Se guardó la clave privada en tu computadora" y además de descarga un fichero .json con las claves. Dale "Cerrar" sobre el emergente. El fichero descargado en mi caso se llama: "ga-master-il3-xxxxxxxxxxx.json" (las  x son un numero de serie, no es importante). Este fichero con las credenciales lo necesitarás para realizar al conexión a la API. 

Finalmente, clica el botón de "Guardar".


## 1.3.2. Google Analytics Reports
El siguiente paso consiste en dar acceso a la cuenta de servicio des de la Vista del Informe de Google Analytics.

En el caso que no tengas creado ninguna vista, o aun no hayas utilizado Google Analytics, a continuación, se explica cómo se crea una vista nueva.

Si por el contrario, ya dispones de una vista de seguimiento de una página de Google Analitics, salta al apartado: **Asignar credenciales a la vista**.



### Crear una nueva Vista

Entra en tu cuenta de Google Analytics des del enlace: https://analytics.google.com/analytics/web/ 

En primer lugar hay que crear un cuenta, para ello clica en el botón de "Configurar" que encontrarás abajo en la izquierda. Clica en la opción de "Crear Cuenta". Da un nombre a tu nueva cuenta de Google Analytics, por ejemplo: "Test IL3" y aprieta la opción de "Siguiente".

Da un nombre a la Propiedad, en este caso se trata de un nombre representativo de la web que quieres monitorizar. Por ejemplo “Web test datos”. Pon la zona horaria, país y moneda (que represente la web).



> **Importante**: En este punto debes apretar el botón de "Opciones avanzadas" que encontrarás justo debajo. Activa el interruptor (situado en la derecha) llamado Crear una propiedad “Universal Analytics”. En continuación necesitaras la URL que contendrá la página que quieres seguir con Google Analytics. SI no tienes ninguna página, puedes crear un blog con en https://www.blogger.com/ y usar al URL de tu blog. Introduce la URL y clica en "Siguiente". SE crearan dos vistas, la universal y la Google Analytics 4.



Aporta la información de web (opcional), dale a "Aceptar".  A continuación, viene el apartad de “Condiciones de Servicio de Google Analytics”. Cambia el país según convenga, marca la RGPD, resto condiciones y botón de "Acepto".

Si todo ha ido bien, ya tienes la nueva Vista asociada a la URL que quieres seguir. Para acceder a la vista, selecciona la nueva cuenta en la pestaña superior: "Test IL3" > "Web test datos" > "Todos los datos de sitios web" > "Abrir". 

Los dos últimos pasos consisten en obtener el ID de la vista y en habilitar el acceso con las credenciales creadas en el apartado de Google Developer Console.

Para ello, dentro de la nueva vista creada, ve a la pestaña de "Administrar" (icono abajo izquierdo). Deberían aparecer 3 columnas: Cuenta, Propiedad, "Todos los datos de sitios web". Clica en el menú "Configuración de la vista" (de la última columna). Copia el identificador bajo el título de "IDE de vista", en mi caso 237871726. Este ID lo necesitaras en la parte de descarga de datos.



### Introducir la etiqueta global en tu sitio web 

Para monitorizar el sitio web deseado, debes insertar un script de seguimiento en la cabecera o sección <HEAD> de tu página web.  En el caso de utilizar Blogger, ve al siguiente apartado.

Para obtener este script es cambiar la Propiedad a la versión Google analytics 4. Para ello, des de Google Analytics, cambia la vista de la cuenta en la pestaña superior: "Test IL3" > "Web test datos – GA4" > "Todos los datos de sitios web" > "Abrir".  Abre la pestaña "Administrar" y en la segunda Columna de propiedad (verás que sólo hay dos columnas), clica de "Flujo de datos" y a continuación abre el flujo que has creado asociado a la página web.

Dentro de este flujo verás que en el partado "Añadir etiquetas en la página", abre el menú asociado a "Etiqueta global de sitio web (gtag.js)".

Aparecerá un script similar al siguiente:

```
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async src="https://www.googletagmanager.com/gtag/js?id=G-ZS336G6MS8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'G-ZS336G6MS8');
</script>
```

Copia este script y pégalo en la cabecera de tu página web.

Debes hacer este paso de pegar el script, tantas veces como paginas tenga tu web.


### Introducir etiqueta global en Blogger.

En el caso de Blogger, una vez hayas creado el Blog dentro de Blogger, selecciona el blog a seguir (Menú superior izquierda). Entra dentro de "Configuración".

En "Opciones básicas", verás el apartado: "Identificador de propiedad de Google Analytics". Clica en el texto e introduce el Identificador de Seguimiento. Este código tiene formato "UA-XXXXXXX-X" y está dentro de la plataforma de Google Analytics, pestaña "Administrar" (abajo izquierda) > segunda Columna "Propiedad" > "Configuración de la propiedad". El código UA-XXXXXX-X qeuda bajo el título de "ID de seguimiento"..


### Asignar credenciales a la vista

Tanto en el caso que hayas creado una vista nueva, como en el que ya tuvieras una disponible, el último paso consiste en asignar las credenciales creadas a la vista o Propiedad.

Accede a la vista mediante la pestaña superior: "Test IL3" > "Web test datos" > "Todos los datos de sitios web" > "Abrir". 

Clica en "Administrar" (abajo izquierda) y clica en el menú "Gestión de usuarios de vistas".

Añade un nuevo usuario con el icono de + (arriba derecha) > Añadir usuarios. 

Pega el correo obtenido en la generación de credenciales en la sección de "Google Developer Console" (también puedes encontrar el correo en el fichero json son las credenciales. En mi caso el correo es: ga-il3-test@ga-master-il3.iam.gserviceaccount.com . Debes deseleccionar la opción de "Notificar a los usuarios nuevos por correo electrónico", ya que no aplica en este caso. Clica en "Añadir".

> **Importante**: Ahora, el tráfico web ya quedará registrado. Puedes comporbarlo en los informes en Tiempo real de Google analytics.


## 1.3.3. Acceso a Google Analytics des de Python

Para acceder a los datos de Google Analytics, vas a necesitar las credenciales que has generado en los apartado anteriores (arxivo: "ga-master-il3-xxxxxxxxxxx.json").

Sube este archivo a Colab (o en tu carpeta local de Jupyter Notebook) a través del icono de Files (menú izquierda) > Icono de "Upload Session Storage" > busca el archivo "ga-master-il3-xxxxxxxxxxx.json" > "Aceptar”

Una vez subido, carga las credenciales con el siguiente código.

Cargamos las librerías.


In [ ]:
import pandas as pd
import numpy as np
import json
import os
from google.colab import drive
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials


Leemos las credenciales.


In [ ]:
# por simplicidad he cambiado el nombre del fichero a "ga-data3.json". 
#cambia este por tu nombre de foichero.

with open('ga-data3.json') as jsonfile:
    KEY_DATA = json.load(jsonfile)


Se define la funciona de obtención de credenciales de Google Analytics.


In [ ]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']

def initialize_analyticsreporting():
  credentials = ServiceAccountCredentials.from_json_keyfile_dict(KEY_DATA, SCOPES)
  analytics = build('analyticsreporting', 'v4', credentials=credentials)
  return analytics


Se cargan las credenciales (¡¡¡paso crítico!!!).


In [ ]:
informe = initialize_analyticsreporting()

type(informe)


La clase anterior dberia ser del tipo : googleapiclient.discovery.Resource.

Si todo ha ido bien, creamos las funciones de obtención del informe y parseado de los resultados.


In [ ]:
# Este código es personalizable

def get_report(analytics, view_id, dimensionVar, startDateVar, endDateVar, pageTokenVar):
  return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': view_id,
          'dateRanges': [{'startDate': startDateVar, 'endDate': 'today'}],
          'metrics': [{'expression': 'ga:entrances'},{'expression': 'ga:pageviews'},{'expression': 'ga:sessions'},{'expression': 'ga:users'}],
          'dimensions': [{'name': dimensionVar}],
          'pageSize': 50000,
          'pageToken': pageTokenVar,
          'samplingLevel': 'LARGE'
        }]
      }
  ).execute()

def handle_report(analytics,view_id, dimensionVar='ga:pagePath', startDateVar='7daysAgo', endDateVar='today', pageTokenVar='0', rows=[]):  
    response = get_report(analytics, view_id, dimensionVar, startDateVar, endDateVar, pageTokenVar)

    #Header, Dimensions Headers, Metric Headers 
    columnHeader = response.get("reports")[0].get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

    #Pagination
    pagetoken = response.get("reports")[0].get('nextPageToken', None)
    
    #Rows
    rowsNew = response.get("reports")[0].get('data', {}).get('rows', [])
    rows = rows + rowsNew
    print("len(rows): " + str(len(rows)))

    #Recursivly query next page
    if pagetoken != None:
        return handle_report(analytics,pagetoken,rows)
    else:
        #nicer results
        nicerows=[]
        for row in rows:
            dic={}
            dimensions = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])

            for header, dimension in zip(dimensionHeaders, dimensions):
                dic[header] = dimension

            for i, values in enumerate(dateRangeValues):
                for metric, value in zip(metricHeaders, values.get('values')):
                    if ',' in value or ',' in value:
                        dic[metric.get('name')] = float(value)
                    else:
                        dic[metric.get('name')] = int(value)
            nicerows.append(dic)
        return nicerows


Copia tu ID de vista en la siguiente variable: ID_VISTA

Recuerda que se indica como obtener este ID en el apartado "Crear una nueva Vista".


In [ ]:
# prueba con tu ID de vista
ID_VISTA = "237871726" 

rows = handle_report(informe,ID_VISTA,dimensionVar='ga:pagePath',startDateVar='7daysAgo',endDateVar='today')
dfanalytics = pd.DataFrame(list(rows))

> **Importante**: Ten en cuenta que si has insertado la etiqueta global en tu sitio web hace poco tiempo, hasta al cabo de unas horas (día siguiente) no apareceran los datos solicitados. Obviamente, también es importante que generes tráfico en tu sitio web.


In [ ]:
dfanalytics


Puedes probar con otras dimensiones, por ejemplo:

- 'ga:browser'
- 'ga:sourceMedium'



In [ ]:
rows = handle_report(informe,ID_VISTA,dimensionVar='ga:sourceMedium',startDateVar='7daysAgo',endDateVar='today')
dfanalytics = pd.DataFrame(list(rows))

In [ ]:
dfanalytics


O incluso cambiando de fechas y ID de vista de otra web donde has habilitado las mismas credenciales.



In [ ]:
ID_VISTA = "30664139"

rows = handle_report(informe,ID_VISTA,dimensionVar='ga:browser',startDateVar='2016-01-01',endDateVar='today')
dfanalytics = pd.DataFrame(list(rows))


In [ ]:
dfanalytics


Para aprender más opciones de reports, métricas y dimeniones de interés, puedes consultar la siguiente referencia:

- https://janakiev.com/blog/python-google-analytics/




## Enlaces de interés

- [Información de la API](https://developers.google.com/analytics?hl=es)
- [Google Analytics Access from Google Colab](https://www.youtube.com/watch?v=msEXSJGMjf4)
- [Getting started with the Google Analytics Reporting API in Python](https://www.themarketingtechnologist.co/getting-started-with-the-google-analytics-reporting-api-in-python/)
- [Submitting your first Google Analytics Reporting API Request](https://medium.com/analytics-for-humans/submitting-your-first-google-analytics-reporting-api-request-cdda19969940)
- [Use the Google Analytics API with Python](http://www.marinamele.com/use-google-analytics-api-with-python)
- [Google Analytics, Python, Pandas, Plotly: Get Started](https://canonicalized.com/google-analytics-python-pandas-plolty)
- [Google Analytics Setup Guide](https://bluerivermountains.com/en/google-analytics-setup)
